In [ ]:
import sys, os
sys.path.append(os.getcwd())

# Just run this block. Please do not modify the following code.
import math
import time
import io
import numpy as np
import csv
from IPython.display import Image

# Pytorch package
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from collections import Counter
from torch.nn.utils.rnn import pad_sequence

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm

# Code provide to you for training and evaluation
# Not sure why this import doesn't work
from utils import train, evaluate, set_seed_nb, plot_curves

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

In [ ]:
# Load in the transcripts and process them into their relevent chunks
from src import context_chunker 
import pandas as pd
from pathlib import Path
import re

# --- Paths ---
path = "data/raw/transcripts"
meta_path = "data/raw/full_test_split.csv"

# --- Load metadata ---
meta_df = pd.read_csv(meta_path, sep=",")
meta_df["Participant_ID"] = meta_df["Participant_ID"].astype(int)
phq_lookup = dict(zip(meta_df["Participant_ID"], meta_df["PHQ_Score"]))

# --- Load transcripts ---
all_transcripts = []
for file in Path(path).glob("*.csv"):
    transcript_num = int(re.search(r"\d+", file.stem).group())

    raw_transcript = context_chunker.read_csv_sentences(file)

    # Some of these transcripts are malformatted
    if not raw_transcript:
        continue

    # Lookup returns -1 if not found - we can quarantine these to a test set later.
    phq_score = phq_lookup.get(transcript_num, -1)
    all_transcripts.append((transcript_num, raw_transcript, phq_score))                           
                           
sequence_len = 512
wordlevel = []
sentencelevel = []
dialoguelevel = []

for _, t, score in all_transcripts:
    wordlevel.append((context_chunker.word_level(t, sequence_len), score))
    sentencelevel.append((context_chunker.sentence_level(t, sequence_len), score))
    dialoguelevel.append((context_chunker.dialogue_level(t, sequence_len), score))

    

In [ ]:
# Sanity check - make sure to clear output after running 

print(wordlevel[0])
print(sentencelevel[0])
print(dialoguelevel[0])